In [1]:
# Let's use Graphiti to create a knowledge graph. In a knowledge graph:
#   - concepts are represented as nodes.
#   - facts are represented as edges (or relationships) connecting those nodes.

In [2]:
from dotenv import load_dotenv
load_dotenv() # load environment variables

True

In [3]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF

In [5]:
# Set up logging and environment variables for connecting to the Neo4j database

# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

# Neo4j connection parameters
# Make sure Neo4j Desktop is running with a local DBMS started
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', "password")

if not neo4j_uri or not neo4j_user or not neo4j_password:
    raise ValueError('NEO4J_URI, NEO4J_USER, and NEO4J_PASSWORD must be set')

In [6]:
# Configure Graphiti to work with Ollama

from graphiti_core import Graphiti
from graphiti_core.llm_client.config import LLMConfig
from graphiti_core.llm_client.openai_client import OpenAIClient
from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient

# Configure Ollama LLM client
llm_config = LLMConfig(
    api_key="abc",  # Ollama doesn't require a real API key
    model="gemma3n",
    small_model="gemma3n",
    base_url="http://localhost:11434/v1",  # Ollama provides this port
)
llm_client = OpenAIClient(config=llm_config)

# Initialize Graphiti with Ollama clients
graphiti = Graphiti(
    neo4j_uri,
    neo4j_user,
    neo4j_password,
    llm_client=llm_client,
    embedder=OpenAIEmbedder(
        config=OpenAIEmbedderConfig(
            api_key="abc",
            embedding_model="nomic-embed-text",
            embedding_dim=768,
            base_url="http://localhost:11434/v1",
        )
    ),
    cross_encoder=OpenAIRerankerClient(client=llm_client, config=llm_config),
)

# Initialize the graph database with graphiti's indices. This only needs to be done once.
await graphiti.build_indices_and_constraints()

2025-08-05 10:47:58 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX relation_uuid IF NOT EXISTS FOR ()-[e:RELATES_TO]-() ON (e.uuid)` has no effect.} {description: `RANGE INDEX relation_uuid FOR ()-[e:RELATES_TO]-() ON (e.uuid)` already exists.} {position: None} for query: 'CREATE INDEX relation_uuid IF NOT EXISTS FOR ()-[e:RELATES_TO]-() ON (e.uuid)'
2025-08-05 10:47:58 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX has_member_uuid IF NOT EXISTS FOR ()-[e:HAS_MEMBER]-() ON (e.uuid)` has no effect.} {description: `RANGE INDEX has_member_uuid FOR ()-[e:HAS_MEMBER]-() ON (e.uuid)` already exists.} {position: None} for query: 'CREATE INDEX has_member_uuid IF NOT EXIST

In [7]:
# Ingest episodes into Graphiti
#   - This will create our knowledge graph in Neo4j (http://localhost:7474/browser/preview/)

# Episodes about Kamala Harris, Gavin Newsom, and the political roles they play in California
#   - Notice that some episodes are structured (JSON) and some are unstructured (raw text)
episodes = [
    {
        'content': 'Kamala Harris is the Attorney General of California. She was previously '
        'the district attorney for San Francisco.',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': 'As AG, Harris was in office from January 3, 2011 – January 3, 2017',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'state': 'California',
            'previous_role': 'Lieutenant Governor',
            'previous_location': 'San Francisco',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
    {
        'content': {
            'name': 'Gavin Newsom',
            'position': 'Governor',
            'term_start': 'January 7, 2019',
            'term_end': 'Present',
        },
        'type': EpisodeType.json,
        'description': 'podcast metadata',
    },
]

# Ingest episodes
for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=f'Freakonomics Radio {i}',
        episode_body=episode['content']
        if isinstance(episode['content'], str)
        else json.dumps(episode['content']),
        source=episode['type'],
        source_description=episode['description'],
        reference_time=datetime.now(timezone.utc),
    )
    print(f'Added episode: Freakonomics Radio {i} ({episode["type"].value})')

2025-08-05 10:48:45 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:48:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:48:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:48:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:48:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:48:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:48:57 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:49:09 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:49:09 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "

Added episode: Freakonomics Radio 0 (text)


2025-08-05 10:49:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:49:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:18 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:24 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:49:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:49:39 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:41 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/complet

Added episode: Freakonomics Radio 1 (text)


2025-08-05 10:49:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:49:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:49:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:50:02 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:16 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:16 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "

Added episode: Freakonomics Radio 2 (json)


2025-08-05 10:50:31 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:31 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:50:41 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:46 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:50:48 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:48 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"
2025-08-05 10:50:50 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:50:52 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/

Added episode: Freakonomics Radio 3 (json)


In [8]:
# Basic Search
#   - Analogous to searching a vector database.
#   - Note that some data has an "invalid_at" date. Data in Graphiti is expected to always be changing
#     and evolving. Facts are true for valid for a time until they're invalidated by more recent facts.

# Perform a hybrid search combining semantic similarity and BM25 retrieval
print("\nSearching for: 'Who was the California Attorney General?'")
results = await graphiti.search('Who was the California Attorney General?')

# Print search results
print('\nSearch Results:')
for result in results:
    print(f'UUID: {result.uuid}')
    print(f'Fact: {result.fact}')
    if hasattr(result, 'valid_at') and result.valid_at:
        print(f'Valid from: {result.valid_at}')
    if hasattr(result, 'invalid_at') and result.invalid_at:
        print(f'Valid until: {result.invalid_at}')
    print('---')

2025-08-05 10:52:26 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Searching for: 'Who was the California Attorney General?'

Search Results:
UUID: 05f3535a-4bfc-44f9-9f3f-b60ac4ca107d
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 17:48:36.319499+00:00
---
UUID: da7fb298-ae23-4096-8f0a-0708a78a7ab3
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 17:48:36.319499+00:00
Valid until: 2025-08-05 17:49:48.478255+00:00
---
UUID: c117f6e1-61f9-4335-ae7f-6df2021813fa
Fact: Gavin Newsom is the Governor of California.
Valid from: 2025-08-05 17:49:48.478255+00:00
---
UUID: 30d029e3-04c2-48d7-9353-ed96e3c88989
Fact: Gavin Newsom is the Governor of California.
Valid from: 2025-08-05 17:50:29.638794+00:00
---
UUID: 2860bc1b-0644-4ed2-bbfe-ad45d02b4c08
Fact: She was previously the district attorney for San Francisco.
Valid from: 2025-08-05 17:48:36.319499+00:00
---
UUID: ff4296fa-9fda-4ff4-be26-dd664feda615
Fact: district attorney for San Francisco
Valid from: 2025-08-05 17:48:36.319499+00:00
---


In [9]:
# Center node search
#   - We can leverage the properties of a knowledge graph to get better search results.
#   - Above, we were retrieving facts about Gavin Newsom when we were expecting facts only on the California
#     Attorney General which is Kamala Harris.
#       - The first result from our basic similarity search ("Kamala Harris is the Attorney General of
#         California") is accurate and closely related to the original query.
#       - We can do a center node search from this "good" result to retrieve facts ranked by their "distance" to
#         Kamala Harris in the knowledge graph. This essentially selects for facts that are relevant to Kamala Harris.

# Use the top search result's UUID as the center node for reranking
if results and len(results) > 0:
    # Get the source node UUID from the top result
    center_node_uuid = results[0].source_node_uuid

    print('\nReranking search results based on graph distance:')
    print(f'Using center node UUID: {center_node_uuid}')

    reranked_results = await graphiti.search(
        'Who was the California Attorney General?', center_node_uuid=center_node_uuid
    )

    # Print reranked search results
    print('\nReranked Search Results:')
    for result in reranked_results:
        print(f'UUID: {result.uuid}')
        print(f'Fact: {result.fact}')
        if hasattr(result, 'valid_at') and result.valid_at:
            print(f'Valid from: {result.valid_at}')
        if hasattr(result, 'invalid_at') and result.invalid_at:
            print(f'Valid until: {result.invalid_at}')
        print('---')
else:
    print('No results found in the initial search to use as center node.')


Reranking search results based on graph distance:
Using center node UUID: d5ae65cb-8f15-4bac-8d0a-8185786d6bae


2025-08-05 11:10:06 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Reranked Search Results:
UUID: 05f3535a-4bfc-44f9-9f3f-b60ac4ca107d
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 17:48:36.319499+00:00
---
UUID: 2860bc1b-0644-4ed2-bbfe-ad45d02b4c08
Fact: She was previously the district attorney for San Francisco.
Valid from: 2025-08-05 17:48:36.319499+00:00
---
UUID: 534bf21a-78a9-497c-a62d-af42e9111b0e
Fact: Kamala Harris was in office from January 3, 2011 – January 3, 2017
Valid from: 2025-08-05 17:49:14.813973+00:00
Valid until: 2017-01-03 00:00:00+00:00
---
UUID: da7fb298-ae23-4096-8f0a-0708a78a7ab3
Fact: Kamala Harris is the Attorney General of California.
Valid from: 2025-08-05 17:48:36.319499+00:00
Valid until: 2025-08-05 17:49:48.478255+00:00
---
UUID: 86bf502e-1ff3-4741-9947-269abe6bddc4
Fact: Gavin Newsom was formerly located in San Francisco.
Valid from: 2025-08-05 17:49:48.478255+00:00
---
UUID: ff4296fa-9fda-4ff4-be26-dd664feda615
Fact: district attorney for San Francisco
Valid from: 2025-08-05 17:48:

In [10]:
# Node Search Using Search Recipes
#   - Up to now, we've been retrieving edges (facts) from the knowledge graph.
#   - Graphiti also allows us to retrieve nodes directly (concepts).

print('\nPerforming node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:')

# Use a predefined search configuration recipe and modify its limit
node_search_config = NODE_HYBRID_SEARCH_RRF.model_copy(deep=True)
node_search_config.limit = 5  # Limit to 5 results

# Execute the node search
node_search_results = await graphiti._search(
    query='California Governor',
    config=node_search_config,
)

# Print node search results
print('\nNode Search Results:')
for node in node_search_results.nodes:
    print(f'Node UUID: {node.uuid}')
    print(f'Node Name: {node.name}')
    node_summary = node.summary[:100] + '...' if len(node.summary) > 100 else node.summary
    print(f'Content Summary: {node_summary}')
    print(f"Node Labels: {', '.join(node.labels)}")
    print(f'Created At: {node.created_at}')
    if hasattr(node, 'attributes') and node.attributes:
        print('Attributes:')
        for key, value in node.attributes.items():
            print(f'  {key}: {value}')
    print('---')

2025-08-05 11:11:25 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/embeddings "HTTP/1.1 200 OK"



Performing node search using _search method with standard recipe NODE_HYBRID_SEARCH_RRF:

Node Search Results:
Node UUID: ecbd3386-9b47-4e09-880b-821cbb2c6d88
Node Name: Governor
Content Summary: Gavin Newsom is the Governor of California. He was previously the Lieutenant Governor and was locate...
Node Labels: Entity
Created At: 2025-08-05 17:49:52.589209+00:00
Attributes:
  labels: ['Entity']
---
Node UUID: ca8da3c2-6acd-4bf2-a13b-75b6ffb73d39
Node Name: California
Content Summary: Gavin Newsom is the Governor of California. He was previously the Lieutenant Governor and was locate...
Node Labels: Entity
Created At: 2025-08-05 17:48:45.648571+00:00
Attributes:
  labels: ['Entity']
---
Node UUID: 60c3bb99-20de-4185-9f20-75702c43da9a
Node Name: Attorney General
Content Summary: Gavin Newsom is the Governor of California. He was previously the Lieutenant Governor and was locate...
Node Labels: Entity
Created At: 2025-08-05 17:48:45.648553+00:00
Attributes:
  labels: ['Entity']
---
Node 